In [1]:
!pip3 install transformers

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 64.2MB/s 
     |████████████████████████████████| 860kB 52.5MB/s 
     |████████████████████████████████| 645kB 64.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=e4f9f34f0d3db4ac100b9e266101fc03f8637796ecc22663ef8ac587bfd47673
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
import numpy as np
import pandas as pd
def get_data(filepath):
	vocab = set()
	genes = []
	labels = []
	max_len = ceil(100000/8)
	with open(filepath, 'r') as f:
		l = f.readline()
		l = f.readline()
		while l != None:
			inside_bracks = re.findall(r'\[(.*?)\]',l)
			if len(inside_bracks) < 2:
				break
			if count == 1000:
				break
			mers = inside_bracks[0].split(", ")
			mers = [mer[1:-1] for mer in mers]
			if len(mers) > max_len:
				max_len = len(mers)
			genes.append(mers)
			gene_labels = inside_bracks[1].split(", ")
			gene_labels = list(map(float, gene_labels)) 
			labels.append(gene_labels)
			vocab.update(mers)
			l = f.readline()
			count += 1
	return vocab, genes, labels, max_len

def read_data(filepath):
	print("reading ",filepath)
	vocab = set()
	genes = []
	labels = []
	df = pd.read_csv(filepath,usecols=[1,2],sep="\t",skiprows=1,header=None)
	print(df.head())
	for entry in df.itertuples():
		kmer_list = [kmer.strip("\'") for kmer in entry[1][1:-1].split(", ")]
		label_list = list(map(float, entry[2][1:-1].split(", ")))
		genes.append(kmer_list)
		labels.append(label_list)
	return genes, labels
	

def pad(genes,labels, max_len):
	print('Padding and creating masks')
	atten_masks = []
	for i in range(len(genes)):
		length = len(genes[i])
		diff = max_len - length
		genes[i].extend(['[PAD]'] *diff)
		labels[i].extend([2.0] *diff)
		mask = [1]*length
		mask.extend([0]*diff)
		atten_masks.append(np.array(mask))
	return np.array(genes),np.array(labels), np.array(atten_masks)

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#call this after padding
#also need to convert to ids
def make_train_batch_data(train_genes_as_ids,train_labels,train_masks):
  print("Making DataLoader")
  print(train_genes_as_ids.shape)
  print(train_labels.shape)
  print(train_masks.shape)
  batch_size=4
  train_inputs = torch.tensor(train_genes_as_ids)
  train_labels = torch.tensor(train_labels)
  train_masks = torch.tensor(train_masks)
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  return train_dataloader


In [0]:

from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.config = TransfoXLConfig(vocab_size_or_config_json_file=len(possible_kmers)+267735, n_heads=8, n_layers=6)
    self.model = TransfoXLModel(self.config)
    self.tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
    self.out_layer = torch.nn.Linear(self.model.d_model, 2)
  def forward(self, input_ids, mems=None):
    hidden_states, mems = self.model(input_ids, mems)
    preds = self.out_layer(hidden_states).squeeze(0)
    return preds, mems


In [7]:

from torch.nn import CrossEntropyLoss
from itertools import product
from math import ceil
#setting some values
window_size = 1012
k=8
possible_kmers = [''.join(c) for c in product('ACTG', repeat=k)]
print(len(possible_kmers))
max_len = ceil(100000/k)
device = torch.device('cuda')

#building model
model = Model()
model.to(device)
model.tokenizer.add_tokens(possible_kmers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
class_weights = torch.tensor(np.array([1.0, 125.0])).float().cuda()
loss_func = CrossEntropyLoss(weight=class_weights)


65536


100%|██████████| 9143613/9143613 [00:00<00:00, 30386850.90B/s]


In [8]:
genes, labels = read_data("/content/drive/My Drive/partitioned_samples/all_samples_8-mer/all_samples_8-mer_train.txt")

reading  /content/drive/My Drive/partitioned_samples/all_samples_8-mer/all_samples_8-mer_train.txt
                                                   1                                                  2
0  ['CGGAGTCT', 'CCCAAGTC', 'CCCGCCGG', 'GCGGGCGC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1  ['GGCGGCGG', 'AAGTGCGC', 'AGCCGCGC', 'GGCATTCT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
2  ['AGTGAGCG', 'ACACAGAG', 'CGGGCCGC', 'CACCGCCG...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
3  ['GGGACCCA', 'GACTTGCC', 'GACCTGTA', 'CGACTCTG...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4  ['AGACGCCC', 'AGCTCGGC', 'CGCCGGGA', 'CCCAGGTA...  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...


In [0]:
from time import time
#how often to print stuff
genes_per_cycle=20
save_path = "/content/drive/My Drive/attentive_splice_weights/8mer-testing-weights.pt"

def save_weights():
  print("Saving weights")
  torch.save(model.state_dict(), save_path)

def print_update(loss_on_cycle,start_time):
  print("Time For {} Genes: {}".format(str(genes_per_cycle),str(time()-start_time)))
  print("Average Loss: " + str(loss_on_cycle / genes_per_cycle))

def run_epoch(genes,labels):
  epoch_loss=0
  cycle_loss = 0
  start_time=time()
  num_samples=len(genes)
  print("Starting next epoch")
  for i in range(num_samples):
    mems = None
    gene_loss = 0
    curr_gene = genes[i]
    curr_labels = labels[i]
    #looping over all windows
    for w in range(0, len(curr_gene), window_size):
      toks = np.array(model.tokenizer.convert_tokens_to_ids(curr_gene[w:w+window_size]))
      window_input_ids = torch.tensor(toks).unsqueeze(0).cuda()
      window_labels = torch.tensor(np.array(curr_labels[w:w+window_size])).long().cuda()
      predictions, mems = model(window_input_ids, mems)
      l = loss_func(predictions, window_labels)
      gene_loss += l.item()
      #adds loss to storage
      l.backward()
    #backpropogates loss
    optimizer.step()
    #reset loss
    optimizer.zero_grad()
    cycle_loss += gene_loss
    if i > 0 and i % genes_per_cycle == 0:
      print_update(cycle_loss,start_time)
      epoch_loss+=cycle_loss
      cycle_loss = 0
      start_time=time()
    if i > 0 and i % 500 == 0:
      save_weights()
  print("Loss on Epoch: "+str(epoch_loss/num_samples))
  save_weights()

def evaluate(genes,labels):
  model.eval()
  num_samples=len(genes)
  total_loss = 0
  print("Starting evaluation")
  for i in range(num_samples):
    mems = None
    curr_gene = genes[i]
    curr_labels = labels[i]
    #looping over all windows
    for w in range(0, len(curr_gene), window_size):
      toks = np.array(model.tokenizer.convert_tokens_to_ids(curr_gene[w:w+window_size]))
      window_input_ids = torch.tensor(toks).unsqueeze(0).cuda()
      window_labels = torch.tensor(np.array(curr_labels[w:w+window_size])).long().cuda()
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        predictions, mems = model(window_input_ids, mems)
        print("Predictions")
        print(list(predictions.detach().cpu().numpy()))
        print("Actual Labels")
        print(curr_labels)
        window_loss = loss_func(predictions,window_labels).item()
        total_loss+=window_loss
  return total_loss/num_samples


In [31]:
#Overfitting to small training sample
training_genes = genes[:10]
training_labels = labels[:10]
testing_genes=genes[10:11]
testing_labels=labels[10:11]
for i in range(0,30):
  print("Step ",i)
  model.train()
  run_epoch(training_genes,training_labels)
  eval_loss=evaluate(testing_genes,testing_labels)
  print(str(eval_loss))



Step  0
Starting next epoch
Loss on Epoch: 0.0
Saving weights
Starting evaluation
Predictions
[array([-0.14254956,  0.372817  ], dtype=float32), array([-0.14254987,  0.37281755], dtype=float32), array([-0.14254995,  0.37281787], dtype=float32), array([-0.14254996,  0.37281767], dtype=float32), array([-0.14254989,  0.37281767], dtype=float32), array([-0.14254975,  0.37281767], dtype=float32), array([-0.14254987,  0.37281767], dtype=float32), array([-0.14254986,  0.37281775], dtype=float32), array([-0.14254983,  0.37281778], dtype=float32), array([-0.14254981,  0.37281784], dtype=float32), array([-0.14254986,  0.3728178 ], dtype=float32), array([-0.14254984,  0.3728177 ], dtype=float32), array([-0.14254992,  0.37281778], dtype=float32), array([-0.14254995,  0.37281784], dtype=float32), array([-0.14255   ,  0.37281796], dtype=float32), array([-0.14255007,  0.37281796], dtype=float32), array([-0.14255004,  0.3728178 ], dtype=float32), array([-0.14254998,  0.37281775], dtype=float32), array

KeyboardInterrupt: ignored

In [8]:
#Brian Training Code
from time import time
genes_per_cycle=50
total_loss = 0
model.train()
start_time=time()
for i in range(len(genes)):
	mems = None
	gene_loss = 0
	for w in range(0, len(genes[i]), window_size):
		toks = np.array(model.tokenizer.convert_tokens_to_ids(genes[i][w:w+window_size]))
		input_ids = torch.tensor(toks).unsqueeze(0).cuda()
		gene_labels = torch.tensor(np.array(labels[i][w:w+window_size])).long().cuda()
		predictions, mems = model(input_ids, mems)
		l = loss_func(predictions, gene_labels)
		gene_loss += l.item()
		#adds loss to storage
		l.backward()
	#backpropogates loss
	optimizer.step()
 	#reset loss
	optimizer.zero_grad()
	total_loss += gene_loss
	if i > 0 and i % genes_per_cycle == 0:
		print("Time For {} Genes: {}".format(str(genes_per_cycle).str(time()-start_time)))
		start_time=time()
		print("Gene: " + str(i) + " loss: " + str(total_loss / genes_per_cycle))
		total_loss = 0
	if i > 0 and i % 500 == 0:
		path = "/content/drive/My Drive/attentive_splice/weights.pt"
		torch.save(model.state_dict(), path)

running on gene: 0
running on gene: 1
running on gene: 2
running on gene: 3
running on gene: 4
running on gene: 5
running on gene: 6
running on gene: 7
running on gene: 8
running on gene: 9
running on gene: 10
Epoch: 10 loss: 6.596543499827385
running on gene: 11
running on gene: 12
running on gene: 13
running on gene: 14
running on gene: 15
running on gene: 16
running on gene: 17
running on gene: 18
running on gene: 19
running on gene: 20
Epoch: 20 loss: 1.113611063361168
running on gene: 21
running on gene: 22
running on gene: 23
running on gene: 24
running on gene: 25
running on gene: 26
running on gene: 27
running on gene: 28
running on gene: 29
running on gene: 30
Epoch: 30 loss: 0.9314092397689819
running on gene: 31
running on gene: 32
running on gene: 33
running on gene: 34
running on gene: 35
running on gene: 36
running on gene: 37
running on gene: 38
running on gene: 39
running on gene: 40
Epoch: 40 loss: 2.26864949464798
running on gene: 41
running on gene: 42
running on gen

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-3b925322d032>", line 15, in <module>
    l.backward()
  File "/usr/local/lib/python3.6/dist-packages/torch/tensor.py", line 166, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py", line 99, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurre

KeyboardInterrupt: ignored

In [0]:
#Ignore
#training loop attemtp at batching
tr_loss=0
for step, batch in enumerate(training_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, b_input_mask)
    print(outputs)
    loss = outputs[0]
    logits = outputs[1] 
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))